In [ ]:
# | default_exp  project.models.bert.data

# Create data and Encoders

> Just run this Notebook to generate the data in `.tmp`

This notebooks creates data to train the BERT in first order logic. It uses `notebooks.utils.prop_logi` module to generates random theorems. Then mask some persentage of tokens.

Generated Data:

```
.tmp
└── data
    ├── training
    │   ├── theorems.json
    │   └── vocab.txt
    └── validation
        ├── theorems.json
        └── vocab.txt
```

The idea is generate a dataset for train and validation, save the datasets on disk `.tmp` and then it is upload it to an s3 buckets to be used by the production pipeline.

During research phase we are going to use the `.tmp` directory to shorcut the process.



In [ ]:
from random import (
    randint,
)

from project.utils.prop_logic import (
    Proof,
)


def generate_random_theorems(
    training_size=10,
    axioms=10,
    teo_steps=100,
    max_teorem_size=1000,
    numbers=1000,
    prepositions=100,
):
    training_set: list = []
    assert numbers >= axioms, "The number of axioms must be less than the number of numbers"
    while len(training_set) < training_size:
        proof = Proof(
            numbers=numbers,
            prepositions=prepositions,
        )

        # Generates Axioms
        for _ in range(
            randint(
                1,
                axioms,
            )
        ):
            proof.append_axiom(proof.get_random_statement())
        proof.generate_steps(
            randint(
                1,
                teo_steps,
            )
        )

        str_proof = str(proof)
        if len(str_proof) < max_teorem_size:
            training_set.append(str_proof)
    return training_set


def generates_vocabulary(
    numbers=1000,
    prepositions=100,
):
    numbers_list = []
    for i in range(
        0,
        numbers,
    ):
        numbers_list.append(str(i))

    prepos = []
    for i in range(
        0,
        prepositions,
    ):
        prepos.append(f"p{i}")

    tokens = [
        "[PAD]",
        "[UNK]",
        "[CLS]",
        "[SEP]",
        "[MASK]",
    ]
    special_word = [
        "nl",
        "hypothesis",
        "conclusion",
        "proof",
    ]
    punctuation = [
        ".",
        ",",
        "[",
        "]",
        "-",
    ]
    logic_operators = [
        "|",
        "&",
        "!",
        "(",
        ")",
        "<",
        ">",
    ]
    rules = [
        "introduction",
        "elimination",
        "axiom",
        "implication",
        "conjution",
        "disjunction",
        "negation",
        "biconditional",
    ]
    generates_vocabulary = tokens + special_word + punctuation + logic_operators + rules + prepos + numbers_list
    return generates_vocabulary


ts = generate_random_theorems(
    numbers=1000,
    prepositions=100,
)

for teo in ts:
    print("----------------")
    print(teo)

In [ ]:
import json
import os

RELATIVE_DATA_PATH = "../.tmp/data"

notebook_path = os.getcwd()
data_dir = os.path.join(
    notebook_path,
    RELATIVE_DATA_PATH,
)
train_dir = os.path.join(
    data_dir,
    "training",
)
val_dir = os.path.join(
    data_dir,
    "validation",
)

In [ ]:
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
if not os.path.exists(val_dir):
    os.makedirs(val_dir)


def save_to_json(
    data,
    filename,
):
    with open(
        filename,
        "w",
    ) as f:
        json.dump(
            data,
            f,
        )


def load_from_json(
    filename,
):
    with open(
        filename,
        "r",
    ) as f:
        data = json.load(f)
    return data


def create_vocab(
    numbers=1000,
    prepositions=100,
):
    vocab = generates_vocabulary(
        numbers=numbers,
        prepositions=prepositions,
    )
    return vocab


def crate_training_data(
    batch_dir,
    batch_size=100,
    max_len=1000,
    numbers=1000,
    prepositions=100,
):
    print("generating_random_theorems()")
    theorems = generate_random_theorems(
        batch_size,
        max_teorem_size=max_len - 2,
        numbers=1000,
        prepositions=100,
    )  # -2, saving space for CLS and END tokens
    print("Making theorems")

    file_name = f"theorems.json"
    file_path = os.path.join(
        batch_dir,
        file_name,
    )
    print(f"Saving to path: {file_path}")
    save_to_json(
        theorems,
        file_path,
    )


max_len = 1024
batch_size = 100000
numbers = 1000
prepositions = 100

# Create Vocabulary
vocab = create_vocab(
    numbers=numbers,
    prepositions=prepositions,
)
with open(
    os.path.join(
        train_dir,
        "vocab.txt",
    ),
    "w",
) as f:
    f.write("\n".join(vocab))
with open(
    os.path.join(
        val_dir,
        "vocab.txt",
    ),
    "w",
) as f:
    f.write("\n".join(vocab))


# # create different batches with different max_pred, to ramp up the difficulty
crate_training_data(
    train_dir,
    max_len=max_len,
    batch_size=batch_size,
    numbers=numbers,
    prepositions=prepositions,
)

# # create validation data
crate_training_data(
    val_dir,
    max_len=max_len,
    batch_size=batch_size * 0.20,
    numbers=numbers,
    prepositions=prepositions,
)